# Quintic Bezier Curves in Each Segment
## Offline Path Planning Optimization

A single cubic Bezier curve is fit within each segment or between two waypoints at a time, and the control points are varied to obtain a path of minimum curvature and curvature variation.

In [1]:
%matplotlib qt
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import scipy.special
from optimparallel import minimize_parallel
from cubic_bezier_planner import calc_bezier_path
from cubic_spline_planner import calc_spline_course

In [2]:
class Path:
    def __init__(self, x, y, yaw, k):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.k = k

## Calculating Quintic Bezier Curve Control Points
A quintic Bezier path requires 6 control points. It has the key advantage that the curvature or likewise the second derivative may be manipulated at the start and endpoints to obtain a curvature continuous path. The offset expresses the distance from each endpoint to middle control points as a fraction of the segment length. The method implemented to derive the 6 control points from a position, first and second derivative vector at the start and endpoints are described in this paper:
http://www2.informatik.uni-freiburg.de/~lau/students/Sprunk2008.pdf

$$
\begin{array}{ll}
P_{1}=\frac{1}{5} t_{s}+P_{0}\\
P_{4}=P_{5}-\frac{1}{5} t_{e}\\
P_{2}=\frac{1}{20} a_{s}+2 P_{1}-P_{0}\\
P_{3}=\frac{1}{20} a_{e}+2 P_{4}-P_{5} .
\end{array}
$$

![](figures/quintic_points.png)

In [3]:

def calc_5ord_bezier_path(sx, sy, sd, sdd, ex, ey, ed, edd, offset_1, offset_2):
    """ Create quintic bezier path by manipulating position, first derivative, second 
    derivative vectors at the start and endpoints """

    sd = np.multiply(sd, 0.2*offset_1)
    sdd = np.multiply(sdd, 0.05)
    ed = np.multiply(ed, 0.2*offset_2)
    edd = np.multiply(edd, 0.05)

    # control points
    p0 = [sx, sy]
    p1 = np.add(p0, sd)

    p2 = np.multiply(p1, 2)
    p2 = np.add(p2, sdd)
    p2 = np.subtract(p2, p0)

    p5 = [ex, ey]
    p4 = np.subtract(p5, ed)

    p3 = np.multiply(p4, 2)
    p3 = np.add(p3, edd)    
    p3 = np.subtract(p3, p5)

    control_points = np.array((p0, p1, p2, p3, p4, p5))
    path = calc_bezier_path(control_points, n_points=170)

    return path, control_points

## Visualization and Base Functions
A spline class is initated with an input set of waypoints and a maximum deviation. Boundary lines are constructed, and this class features functions for calculating derivatives, yaw, curvature and path distance. The method for visualizing the outputs are described

In [4]:
class Spline():

    def __init__(self, ax, ay, bound, wheelbase=2.5, max_steer=0.95, max_curvature=0.18):
        
        #input waypoint coordinates
        ayaw, k = self.calc_bisector_yaw_curvature(ax, ay)
        self.waypoints = Path(ax, ay, ayaw, k)

        self.wheelbase = wheelbase
        self.max_steer = max_steer
        self.max_curvature = max_curvature
        
        # defines and sets left and right boundary lines
        self.bound = bound
        lax, lay, rax, ray = self.init_boundary()
        self.left_bound = Path(lax, lay, None, None)
        self.right_bound = Path(rax, ray, None, None)

        # default unoptimized cubic bezier path to initialize curvature
        bx, by, ctr_pt_x, ctr_pt_y = self.quintic_bezier_path(ax, ay, [False])
        byaw, bk = self.calc_yaw_curvature(bx, by)
        self.default_path = Path(bx, by, byaw, bk)

        # default seeding path for cubic spline
        cx, cy, cyaw, ck, _ = calc_spline_course(ax, ay, 0.5)
        self.seeding_path = Path(cx, cy, cyaw, ck)

        self.optimized_path = Path([], [], [], [])
        self.ctr_points = Path(ctr_pt_x, ctr_pt_y, [], [])

    def calc_d(self, x, y):
        """ Calculates the first derivatives of the input arrays """

        dx, dy = [], []

        for i in range(0, len(x)-1):
            dx.append(x[i+1] - x[i])
            dy.append(y[i+1] - y[i])
        
        dx.append(dx[-1])
        dy.append(dy[-1])
        return dx, dy

    def calc_yaw_curvature(self, x, y):
        """ Calculates the yaw and curvature given an input path """

        dx, dy = self.calc_d(x,y)
        ddx, ddy = self.calc_d(dx, dy)
        yaw = []
        k = []

        for i in range(0, len(x)):
            yaw.append(math.atan2(dy[i], dx[i]))
            k.append( (ddy[i] * dx[i] - ddx[i] * dy[i]) / ((dx[i]**2 + dy[i]**2)**(3/2)) )
    
        return yaw, k

    def calc_path_dist(self, x, y):
        """ Calculates the total path distance """

        dx, dy = self.calc_d(x, y)
        dx = np.absolute(dx)
        dy = np.absolute(dy)
        ddist = np.hypot(dx, dy)

        return np.sum(ddist)
    
    def calc_bisector_yaw_curvature(self, x, y):

        dx, dy = self.calc_d(x,y)
        ddx, ddy = self.calc_d(dx, dy)
        yaw, k = [], []

        # curvature calculation
        for i in range(0, len(x)):
            k.append( (ddy[i] * dx[i] - ddx[i] * dy[i]) / ((dx[i]**2 + dy[i]**2)**(3/2)) )
    
        # firspoint yaw unchanged
        yaw.append(math.atan2(dy[0], dx[0]))

        for i in range(1, len(x)-1):
            
            #first derivative unit vectors at each point
            d0 = np.array( [dx[i], dy[i]] )
            d0_hat = d0 /  np.linalg.norm(d0)
            d1 = np.array( [dx[i-1], dy[i-1]] )
            d1_hat = d1 / np.linalg.norm(d1)
            d_bisect = d0_hat + d1_hat
            yaw.append(math.atan2(d_bisect[1], d_bisect[0]))
        
        #lastpoint yaw unchanged
        yaw.append(math.atan2(dy[-1], dx[-1]))
        return yaw, k

    def init_boundary(self):
        """ Determines the position of boundary lines for visualization """

        # initialize headings
        rax, ray, lax, lay = [], [], [], []

        for n in range(0, len(self.waypoints.yaw)):
            lax.append(self.waypoints.x[n] - self.bound*np.sin(self.waypoints.yaw[n]))
            lay.append(self.waypoints.y[n] + self.bound*np.cos(self.waypoints.yaw[n]))
            rax.append(self.waypoints.x[n] + self.bound*np.sin(self.waypoints.yaw[n]))
            ray.append(self.waypoints.y[n] - self.bound*np.cos(self.waypoints.yaw[n]))
        
        return lax, lay, rax, ray

In [5]:
def visualize(spline, input_only=False):
    """ Function to visualize present output """

    if input_only==True:
        plt.subplots(1)
        plt.plot(spline.left_bound.x, spline.left_bound.y, '--r', alpha=0.5, label="left boundary")
        plt.plot(spline.right_bound.x, spline.right_bound.y, '--g', alpha=0.5, label="right boundary")
        plt.plot(spline.default_path.x, spline.default_path.y, '-y', label="default")
        plt.plot(spline.waypoints.x, spline.waypoints.y, '.', label="waypoints")
        
        for n in range(len(spline.waypoints.x)):
            if n%5 == 0:
                plt.annotate(n, (spline.waypoints.x[n], spline.waypoints.y[n]), alpha=0.5)

        plt.grid(True)
        plt.legend()
        plt.axis("equal")
        plt.show()
        return

    plt.subplots(1)
    plt.plot(spline.left_bound.x, spline.left_bound.y, '--r', alpha=0.5, label="left boundary")
    plt.plot(spline.right_bound.x, spline.right_bound.y, '--g', alpha=0.5, label="right boundary")
    plt.plot(spline.default_path.x, spline.default_path.y, '-y', label="default")
    plt.plot(spline.optimized_path.x, spline.optimized_path.y, '-m', label="optimized")
    plt.plot(spline.ctr_points.x, spline.ctr_points.y, 'xr', alpha=0.2, label="control points")
    plt.plot(spline.waypoints.x, spline.waypoints.y, '.', label="waypoints")
    plt.grid(True)
    plt.legend()
    plt.title("Path")
    plt.axis("equal")

    # Heading plot
    plt.subplots(1)
    plt.plot([np.rad2deg(iyaw) for iyaw in spline.default_path.yaw], "-y", label="original")
    plt.plot([np.rad2deg(iyaw) for iyaw in spline.optimized_path.yaw], "-m", label="optimized")
    plt.grid(True)
    plt.legend()
    plt.title("Heading")
    plt.xlabel("line length[m]")
    plt.ylabel("yaw angle[deg]")

    # Curvature plot
    plt.subplots(1)
    plt.plot(spline.default_path.k, "-y", label="original")
    plt.plot(spline.optimized_path.k, "-m", label="optimized")
    plt.grid(True)
    plt.legend()
    plt.title("Curvature")
    plt.xlabel("line length[m]")
    plt.ylabel("curvature [1/m]")

    plt.show()

## Determining First Derivative Vector
The heading is formed as the normal to the bisector of the angle formed at each waypoint. The target heading at the start and end waypoint are fixed as is. The first derivative vector is obtained from the target heading and the less of the distances to the next or previous waypoint.
https://users.soe.ucsc.edu/~elkaim/Documents/camera_WCECS2008_IEEE_ICIAR_58.pdf  

$$
\begin{array}{l}
\alpha_{i}=\angle p_{i-1} p_{i}+\frac{\angle p_{i} p_{i+1}-\angle p_{i-1} p_{i}}{2} \\
l_{i}=\min \left(\left\|p_{i-1}-p_{i}\right\|,\left\|p_{i}-p_{i+1}\right\|\right) \\
p_{i}^{\prime}=l_{i} *\left[\begin{array}{c}
\cos \left(\alpha_{i}\right) \\
\sin \left(\alpha_{i}\right)
\end{array}\right]
\end{array}
$$

In [6]:
class Spline(Spline):

    def calc_1st_derivative_heuristic(self, ax, ay):
        """ Approximates the first derivative vector at all the waypoints """

        ayaw, _ = self.calc_bisector_yaw_curvature(ax, ay)
        n_points = len(ax)
        dx, dy = [], []

        for n in range(n_points):

            if n == 0:
                dist = np.hypot( (ax[n+1] - ax[n]), (ay[n+1] - ay[n]) )
            elif n == n_points-1:
                dist = np.hypot( (ax[n-1] - ax[n]), (ay[n-1] - ay[n]) )

            # for all points except first and last, the tangent length is
            # minimum of the next and previous segment length
            else: 
                d0 = np.hypot( (ax[n-1] - ax[n]), (ay[n-1] - ay[n]) )
                d1 = np.hypot( (ax[n+1] - ax[n]), (ay[n+1] - ay[n]) )
                dist = min(d0, d1)

            dx.append(dist * np.cos(ayaw[n]))
            dy.append(dist * np.sin(ayaw[n]))

        return dx, dy

## Determining Second Derivative Vector
The second derivative vector, which equates to the curvature conditions at each waypoint is determined by first interpolating a set of cubic Bezier curves from the first derivative vector. This results in a curvature discontinuity, the weighted average of the endpoint second derivative conditions is taken to determine the second derivative vector at the waypoint.
http://www2.informatik.uni-freiburg.de/~lau/students/Sprunk2008.pdf

![](figures/heuristic.png) 

Based on the first derivative vectors obtained at each waypoint, the start and endpoint second derivative vectors for consecutive cubic Bezier curve may be described as follows
$$
S_{A B}^{\prime \prime}(1)=6\left(\left(A+\frac{1}{3} t_{A}\right)-2\left(B-\frac{1}{3} t_{B}\right)+B\right)=6 A+2 t_{A}+4 t_{B}-6 B \\
S_{B C}^{\prime \prime}(0)=6\left(B-2\left(B+\frac{1}{3} t_{B}\right)+\left(C-\frac{1}{3} t_{C}\right)\right)=-6 B-4 t_{B}-2 t_{C}+6 C \\
$$
Based on the length of the previous and next segment, a weighted average is taken of the endpoint curvatures as follow &nbsp;
$$
a=\alpha S_{A B}^{\prime \prime}(1)+\beta S_{B C}^{\prime \prime}(0)=\alpha\left(6 A+2 t_{A}+4 t_{B}-6 B\right)+\beta\left(-6 B-4 t_{B}-2 t_{C}+6 C\right),\\
\alpha=\frac{d_{B C}}{d_{A B}+d_{B C}} \quad \beta=\frac{d_{A B}}{d_{A B}+d_{B C}}
$$

In [7]:
class Spline(Spline):

    def calc_2nd_derivative_heuristic(self, ax, ay, dx, dy):
        """ Approximates the second derivative vector at all waypoints """

        n_points = len(ax)
        ddx, ddy = [], []

        for n in range(n_points):

            # for startpoint, start conditions of first bezier curve
            if n == 0:
                b = np.array([ax[n], ay[n]])
                c = np.array([ax[n+1], ay[n+1]])
                tb = np.array([dx[n], dy[n]])
                tc = np.array([dx[n+1], dy[n+1]])
                dd_vect = -6.0*b - 4.0*tb - 2.0*tc + 6.0*c
                
            # for endpoint, take end conditions of last bezier curve
            elif n == n_points - 1:
                a = np.array([ax[n-1], ay[n-1]])
                b = np.array([ax[n], ay[n]])
                ta = np.array([dx[n-1], dy[n-1]])
                tb = np.array([dx[n], dy[n]])
                dd_vect = 6.0*a + 2.0*ta + 4.0*tb - 6.0*b

            # for all points in between, take weighted average at discontinuity
            else:
                a = np.array([ax[n-1], ay[n-1]])
                b = np.array([ax[n], ay[n]])
                c = np.array([ax[n+1], ay[n+1]])

                ta = np.array([dx[n-1], dy[n-1]])
                tb = np.array([dx[n], dy[n]])
                tc = np.array([dx[n+1], dy[n+1]])

                # determining distance for calculating weights
                ab = a-b
                bc = b-c
                ab = np.hypot(ab[0], ab[1])
                bc = np.hypot(bc[0], bc[1])
                alpha = bc / (ab + bc)
                beta = ab / (ab + bc)

                dd_vect = alpha*(6.0*a + 2.0*ta + 4.0*tb - 6.0*b) + beta*(-6.0*b - 4.0*tb - 2.0*tc + 6.0*c)

            ddx.append(dd_vect[0])
            ddy.append(dd_vect[1])

        return ddx, ddy

## Fitting Quintic Bezier Curves in Each Segment
The first and second derivative conditions at each waypoint are determined as vectors with magnitude varying with segment length. Then, a single quintic bezier curve is fit into each segment within these conditions.

![](figures/segments.png)

In [8]:
class Spline(Spline):

    def quintic_bezier_path(self, ax, ay, offset):
        """ Piecewise interpolation of quintic bezier curves through input waypoints """

        if any(offset) == False:
            offset = np.ones(len(ax))

        dyaw, _ = self.calc_yaw_curvature(ax, ay)
        
        # control point and path array
        cx, cy, ctr_pt_x, ctr_pt_y = [], [], [], []

        # obtain target second derivative
        dx, dy = self.calc_1st_derivative_heuristic(ax, ay)
        ddx, ddy = self.calc_2nd_derivative_heuristic(ax, ay, dx, dy)

        # for n waypoints, there are n-1 bezier curves
        for i in range(len(ax)-1):

            path, points = calc_5ord_bezier_path( ax[i], ay[i], np.array( [dx[i], dy[i]] ), np.array( [ddx[i], ddy[i]] ), 
                                                ax[i+1], ay[i+1], np.array( [dx[i+1], dy[i+1]] ), np.array( [ddx[i+1], ddy[i+1]] ),
                                                offset[i], offset[i+1] )

            cx = np.concatenate((cx, path.T[0][:-1]))
            cy = np.concatenate((cy, path.T[1][:-1]))
            
            for p in points:
                ctr_pt_x.append( p[0] )
                ctr_pt_y.append( p[1] )

        return cx, cy, ctr_pt_x, ctr_pt_y

## Approach 1: Optimizing Execution Time with Steering constraints
The original paper proposed the following cost function, intending to miinimize the execution time $t$ while being able to abide by the lateral offset $d_{\max }$ and maximum steering angle $\varphi_{\max }$ constraints. The constraints are imposed by a cost which returns an exponentially greater value when greater than the limit.
$$
E=t+\sum_{j=1}^{n}\left(\text { penalty }\left(\frac{\left\|\varphi_{j}\right\|}{\varphi_{\max }}\right)+\text { penalty }\left(\frac{\left\|d_{j}\right\|}{d_{\max }}\right)\right)
$$
$$
\text { penalty }(c)=\exp (25 \cdot(c-0.9))
$$
The steering profile throughout the path is approximated from the vehicle wheelbase $l_\text {vehicle }$ and the curvature $k_{j}$ at each point.
$$
\varphi_{j}=\arctan \left(l_{\text {vehicle }} \cdot k_{j}\right)
$$


In [9]:
class Spline(Spline):
    
    def deviation_penalty(self, dev, max_dev):
        """ Penalty to return exponential cost when deviation is greater than maximum """
        dev = np.array(dev)
        dev = np.absolute(dev) / max_dev
        penalty = np.exp( 25 * (dev - 0.9) )
        return np.sum(penalty)

   
    def steering_penalty(self, k, max_steer):
        """ Penalty to calculating steering angle profile throughout path, returns
        exponential cost when greater than vehicle limits"""
        k = np.array(k)
        steer_profile  = np.arctan(self.wheelbase * k)
        steer_cons_ratio = np.absolute(steer_profile) / max_steer
        penalty = np.exp( 25 * (steer_cons_ratio - 0.9) )
        return np.sum(penalty)

    def quintic_objective_func_1(self, params):
        """ Objective function for interpolating piecewise quintic bezier path """

        ax = self.waypoints.x.copy()
        ay = self.waypoints.y.copy()

        # calculate offsets and input waypoints
        waypoints = len(self.waypoints.yaw)
        deviation_lat = params[ :(waypoints-2) ]
        offset = params[ (waypoints-2): ]

        for n in range(0, len(self.waypoints.yaw)-2):
            ax[n+1] -= deviation_lat[n]*np.sin(self.waypoints.yaw[n+1])
            ay[n+1] += deviation_lat[n]*np.cos(self.waypoints.yaw[n+1])

        bx, by, _, _ = self.quintic_bezier_path(ax, ay, offset)
        yaw, k = self.calc_yaw_curvature(bx, by)

        # constraints
        steering_constraint = self.steering_penalty(k, self.max_steer)
        deviation_constraint = self.deviation_penalty(deviation_lat, self.bound)

        # cost of curvature continuity
        dk, _ = self.calc_d(k, k)
        absolute_dk = np.square(dk)
        continuity_cost = 5000 * np.mean(absolute_dk)  

        # curvature cost
        absolute_k = np.square(k)
        curvature_cost = 1 * np.mean(absolute_k)

        return 0.01*self.calc_path_dist(bx, by) + continuity_cost + 0.001*deviation_constraint + steering_constraint

    def optimize_min_quintic_1(self):
        """ Minimize quintic objective function using SciPy solver """

        print("Attempting optimization minima")

        # create bounds for waypoint deviation and tangent length
        bnds = []
        for n in range(len(self.waypoints.yaw)-2):
            bnds.append((-1.5*self.bound, 1.5*self.bound))
        for n in range(len(self.waypoints.yaw)):
            bnds.append((0, 1.3))
        bnds = tuple(bnds)

        # create initial guess for deviation and tangent lengths
        initial_guess = []
        for n in range(len(self.waypoints.yaw)-2):
            initial_guess.append(0.0)
        for n in range(len(self.waypoints.yaw)):
            initial_guess.append(0.2)

        result = minimize_parallel(self.quintic_objective_func_1, initial_guess, bounds=bnds)

        ax = self.waypoints.x.copy()
        ay = self.waypoints.y.copy()

        if result.success:
            print("optimized true")
            params = result.x
            
            # collects offsets for individual bezier curves
            waypoints = len(self.waypoints.yaw)
            deviation_lat = params[ :(waypoints-2) ]
            offset = params[ (waypoints - 2): ]
      

            # updated set of waypoints
            for n in range(0, len(self.waypoints.yaw)-2):
                ax[n+1] -= deviation_lat[n]*np.sin(self.waypoints.yaw[n+1])
                ay[n+1] += deviation_lat[n]*np.cos(self.waypoints.yaw[n+1])

            x, y, ctr_pt_x, ctr_pt_y = self.quintic_bezier_path(ax, ay, offset)
            yaw, k = self.calc_yaw_curvature(x, y)

            # update path optimized path and control points
            self.optimized_path = Path(x, y, yaw, k)
            self.ctr_points = Path(ctr_pt_x, ctr_pt_y, [], [])        

        else:
            print("optimization failure, defaulting")
            exit()

## Approach 2: Optimizing Minimum Curvature and Curvature Continuity
The second approach intends to minimize curvature and curvature continuity. This objective function allowed the program to much quickly find a solution, with only a boundary constraint.
$$
J_{i}=\int_{t_{i-1}}^{t_{i}}\left[a_{i}\left|\kappa_{i}(t)\right|^{2}+b_{i}\left|\dot{\kappa_{i}}(t)\right|^{2}\right] d t
$$

This method may be incorperated with a curvature constraint using a user defined SciPy inequality constraint, at the cost of significantly poorer computation time.

In [10]:
class Spline(Spline):

    def quintic_objective_func_2(self, params):
        """ Objective function for interpolating piecewise quintic bezier path """

        ax = self.waypoints.x.copy()
        ay = self.waypoints.y.copy()

        # calculate offsets and input waypoints
        waypoints = len(self.waypoints.yaw)
        deviation_lat = params[ :(waypoints-2) ]
        offset = params[ (waypoints-2): ]

        for n in range(0, len(self.waypoints.yaw)-2):
            ax[n+1] -= deviation_lat[n]*np.sin(self.waypoints.yaw[n+1])
            ay[n+1] += deviation_lat[n]*np.cos(self.waypoints.yaw[n+1])

        bx, by, _, _ = self.quintic_bezier_path(ax, ay, offset)
        yaw, k = self.calc_yaw_curvature(bx, by)

        # cost of curvature continuity
        dk, _ = self.calc_d(k, k)
        absolute_dk = np.square(dk)
        continuity_cost = 40.0 * np.mean(absolute_dk)  

        # curvature cost
        absolute_k = np.square(k)
        curvature_cost = 1.0 * np.mean(absolute_k)

        return curvature_cost

    def optimize_min_quintic_2(self, constrained=False):
        """ Minimize quintic objective function using SciPy solver """
        
        print("Attempting optimization minima")

        # create bounds for waypoint deviation and tangent length
        bnds = []
        for n in range(len(self.waypoints.yaw)-2):
            bnds.append((-self.bound, self.bound))
        for n in range(len(self.waypoints.yaw)):
            bnds.append((0, 1.3))
        bnds = tuple(bnds)

        # create initial guess for deviation and tangent lengths
        initial_guess = []
        for n in range(len(self.waypoints.yaw)-2):
            initial_guess.append(0.0)
        for n in range(len(self.waypoints.yaw)):
            initial_guess.append(0.2)

        if constrained:
            max_curvature = {'type': 'ineq', 'fun': self.max_curvature_constraint}
            constraints = [max_curvature]
            result = optimize.minimize(self.quintic_objective_func_2, initial_guess, bounds=bnds, constraints=constraints)
        else:
            result = minimize_parallel(self.quintic_objective_func_2, initial_guess, bounds=bnds)


        ax = self.waypoints.x.copy()
        ay = self.waypoints.y.copy()

        if result.success:
            print("optimized true")
            params = result.x
            
            # collects offsets for individual bezier curves
            waypoints = len(self.waypoints.yaw)
            deviation_lat = params[ :(waypoints-2) ]
            offset = params[ (waypoints - 2): ]
      

            # updated set of waypoints
            for n in range(0, len(self.waypoints.yaw)-2):
                ax[n+1] -= deviation_lat[n]*np.sin(self.waypoints.yaw[n+1])
                ay[n+1] += deviation_lat[n]*np.cos(self.waypoints.yaw[n+1])

            x, y, ctr_pt_x, ctr_pt_y = self.quintic_bezier_path(ax, ay, offset)
            yaw, k = self.calc_yaw_curvature(x, y)

            # update path optimized path and control points
            self.optimized_path = Path(x, y, yaw, k)
            self.ctr_points = Path(ctr_pt_x, ctr_pt_y, [], [])        

        else:
            print("optimization failure, defaulting")
            exit()

    def max_curvature_constraint(self, params):
        """ Optional method to impose curvature constraint using SciPy """

        ax = self.waypoints.x.copy()
        ay = self.waypoints.y.copy()

        # calculate offsets and input waypoints
        waypoints = len(self.waypoints.yaw)
        deviation_lat = params[ :(waypoints-2) ]
        offset = params[ (waypoints-2): ]

        for n in range(0, len(self.waypoints.yaw)-2):
            ax[n+1] -= deviation_lat[n]*np.sin(self.waypoints.yaw[n+1])
            ay[n+1] += deviation_lat[n]*np.cos(self.waypoints.yaw[n+1])
        
        bx, by, _, _ = self.quintic_bezier_path(ax, ay, offset)
        yaw, k = self.calc_yaw_curvature(bx, by)
        abs_k = np.absolute(k)
        max_k = np.max(abs_k)

        return self.max_curvature - max_k

In [11]:
def path_planning(ax, ay, boundary, n_point=7):
    """ Function to conduct path planning optimization n_points at a time """

    x, y , yaw, k = [], [], [], []
    segments = [0, 5, 10, 16, 25, 35, 42, 48, 53, 62, 67, 74, 81, 87, 93, 102, 110]
    
    
    for s in range(1, len(segments)):

        print("Optimizing segment ", s, " of ", len(segments)+1)

        sx = ax [ segments[s-1] : segments[s] ]
        sy = ay [ segments[s-1] : segments[s] ]

        spline = Spline(sx, sy, boundary)
        %time spline.optimize_min_quintic_2(constrained=False)

        x = np.concatenate(( x, spline.optimized_path.x ))
        y = np.concatenate(( y, spline.optimized_path.y ))
        yaw = np.concatenate(( yaw, spline.optimized_path.yaw ))    
        k = np.concatenate(( k, spline.optimized_path.k ))
    
    path = Spline(ax, ay, boundary)
    path.optimized_path.x = x
    path.optimized_path.y = y
    path.optimized_path.yaw = yaw
    path.optimized_path.k = k

    return path

In [12]:
import pandas as pd
dir_path = 'waypoints/ngeeann2.0.csv'
df = pd.read_csv(dir_path)
ax = df['x'].values.tolist()
ay = df['y'].values.tolist()
ax, ay = ax[0:115], ay[0:115]
boundary = 0.5
# spline = Spline(ax, ay, boundary)
# visualize(spline, input_only=True)

path = path_planning(ax, ay, boundary)
visualize(path)


Optimizing segment  1  of  18
Attempting optimization minima
optimized true
CPU times: user 879 ms, sys: 38.4 ms, total: 917 ms
Wall time: 4.7 s
Optimizing segment  2  of  18
Attempting optimization minima
optimized true
CPU times: user 765 ms, sys: 52.3 ms, total: 817 ms
Wall time: 4 s
Optimizing segment  3  of  18
Attempting optimization minima
optimized true
CPU times: user 1.21 s, sys: 42.4 ms, total: 1.25 s
Wall time: 5.67 s
Optimizing segment  4  of  18
Attempting optimization minima
optimized true
CPU times: user 8.09 s, sys: 129 ms, total: 8.22 s
Wall time: 39.1 s
Optimizing segment  5  of  18
Attempting optimization minima
optimized true
CPU times: user 12.5 s, sys: 154 ms, total: 12.6 s
Wall time: 59.6 s
Optimizing segment  6  of  18
Attempting optimization minima
optimized true
CPU times: user 5.69 s, sys: 107 ms, total: 5.8 s
Wall time: 28.3 s
Optimizing segment  7  of  18
Attempting optimization minima
optimized true
CPU times: user 715 ms, sys: 45.5 ms, total: 760 ms
Wall